<a href="https://colab.research.google.com/github/swapnilbetkar/python/blob/Dev_Class/Code_CA2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
from google.colab import drive
drive.mount('/content/drive') 



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [10]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import SGDRegressor
from sklearn.svm import SVR
import plotly.graph_objs as go
import plotly.figure_factory as ff
import warnings
warnings.filterwarnings('ignore')

In [11]:
dataset = pd.read_csv("/content/drive/MyDrive/Datasets/SuperMart.csv")
pd.set_option('display.max_columns', None) # to make sure you can see all the columns in output window
print(dataset.head())
print(dataset.shape)
print(dataset.info())
print(dataset.describe())


    Education Age_Group Marital_Status Income_Group  Dependents  \
0  Graduation     30-60       Divorced      60K-89K           0   
1  Graduation     30-60         Single      30K-59K           0   
2  Graduation     30-60        Married      60K-89K           1   
3  Graduation     30-60       Together      30K-59K           2   
4  Graduation       <30         Single         <30K           1   

   NumDealsPurchases  NumWebPurchases  NumStorePurchases  AcceptedCmp1  \
0                  1                8                  6             0   
1                  1               10                  7             0   
2                  1                5                  5             0   
3                  1                1                  2             0   
4                  2                4                  2             0   

   AcceptedCmp2  AcceptedCmp3  AcceptedCmp4  AcceptedCmp5  AcceptedCmp6  \
0             0             0             0             0             1   
1 

In [12]:
# Converting Categorical features into Numerical features
categorical_features = ['Marital_Status']

#Using map function for converting categorcal features to numerical features
dataset['Age_Group'] = dataset['Age_Group'].map({'>60':2,'30-60':1,'<30' :0})
dataset['Income_Group'] = dataset['Income_Group'].map({'>=90K':3,'60K-89K':2,'30K-59K' :1,'<30K':0})
dataset['Education'] = dataset['Education'].map({'PhD':4,'Master':3,'Graduation' :2,'High School':1,'Middle School':0})
final_data = pd.get_dummies(dataset, columns = categorical_features)
print(final_data.info())
print(final_data.head(2))


# Plotting Correlation Heatmap
corrs = dataset.corr()
figure = ff.create_annotated_heatmap(
    z=corrs.values,
    x=list(corrs.columns),
    y=list(corrs.index),
    annotation_text=corrs.round(2).values,
    showscale=True)
figure.show()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2240 entries, 0 to 2239
Data columns (total 19 columns):
 #   Column                   Non-Null Count  Dtype
---  ------                   --------------  -----
 0   Education                2240 non-null   int64
 1   Age_Group                2240 non-null   int64
 2   Income_Group             2240 non-null   int64
 3   Dependents               2240 non-null   int64
 4   NumDealsPurchases        2240 non-null   int64
 5   NumWebPurchases          2240 non-null   int64
 6   NumStorePurchases        2240 non-null   int64
 7   AcceptedCmp1             2240 non-null   int64
 8   AcceptedCmp2             2240 non-null   int64
 9   AcceptedCmp3             2240 non-null   int64
 10  AcceptedCmp4             2240 non-null   int64
 11  AcceptedCmp5             2240 non-null   int64
 12  AcceptedCmp6             2240 non-null   int64
 13  Complain                 2240 non-null   int64
 14  Revenue                  2240 non-null   int64
 15  Mari

In [13]:
# Dividing dataset into label and feature sets
X = final_data.drop(['Revenue','Income_Group'], axis = 1) # Features
Y = final_data['Revenue'] # Labels
print(type(X))
print(type(Y))
print(X.shape)
print(Y.shape)

<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.series.Series'>
(2240, 17)
(2240,)


In [14]:
# Normalizing numerical features so that each feature has mean 0 and variance 1
feature_scaler = StandardScaler()
X_scaled = feature_scaler.fit_transform(X)

In [17]:
#Implementing Linear Regression
# Tuning the SGDRegressor parameters 'eta0' (learning rate) and 'max_iter' using Grid Search
# sgdr = SGDRegressor(random_state = 1)
# grid_param = {'eta0': [0.00001,.0001, .001], 'max_iter':[10000, 20000]}

# gd_sr = GridSearchCV(estimator=sgdr, param_grid=grid_param, scoring='r2', cv=5)

# gd_sr.fit(X_scaled, Y)

# best_parameters = gd_sr.best_params_
# print(best_parameters)

# best_result = gd_sr.best_score_ # Mean cross-validated score of the best_estimator
# print(best_result)

#Building SGDRegressor using the tuned parameters
# sgdr = SGDRegressor(eta0=.0001, max_iter=10000, penalty=None, random_state=1)
# sgdr.fit(X_scaled,Y)
# print('Intercept', sgdr.intercept_)
# print(pd.DataFrame(zip(X.columns, sgdr.coef_), columns=['Features','Coefficients']).sort_values(by=['Coefficients'],ascending=False))

Intercept [605.79853167]
                   Features  Coefficients
4           NumWebPurchases    291.831135
5         NumStorePurchases    164.838875
10             AcceptedCmp5    107.256675
6              AcceptedCmp1     41.714178
11             AcceptedCmp6     22.280110
9              AcceptedCmp4     16.178665
0                 Education     14.998273
1                 Age_Group      5.258141
16  Marital_Status_Together      4.698726
15    Marital_Status_Single      3.326094
13  Marital_Status_Divorced     -0.081053
8              AcceptedCmp3     -1.510914
7              AcceptedCmp2     -3.544246
12                 Complain     -6.996711
14   Marital_Status_Married     -7.143543
3         NumDealsPurchases    -27.418560
2                Dependents    -86.840366


In [21]:
# Implementing L2 Regularization (Ridge Regression)
# Tuning Regularization parameter alpha
# sgdr = SGDRegressor(eta0=.0001, max_iter=10000, penalty='l2', random_state=1)
# grid_param = {'alpha': [0.00001,.0001, .001, .01, .1, 1]}

# gd_sr = GridSearchCV(estimator=sgdr, param_grid=grid_param, scoring='r2', cv=5)

# gd_sr.fit(X_scaled, Y)

# best_parameters = gd_sr.best_params_
# print(best_parameters)

# best_result = gd_sr.best_score_ # Mean cross-validated score of the best_estimator
# print(best_result)

{'alpha': 0.001}
0.7633223737240253


In [22]:
# Building SGDRegressor using the tuned parameters
# sgdr = SGDRegressor(eta0=.0001, max_iter=10000, penalty='l2', alpha=0.001, random_state=1)
# sgdr.fit(X_scaled,Y)
# print('Intercept', sgdr.intercept_)
# print(pd.DataFrame(zip(X.columns, sgdr.coef_), columns=['Features','Coefficients']).sort_values(by=['Coefficients'],ascending=False))

Intercept [605.79452965]
                   Features  Coefficients
4           NumWebPurchases    290.505507
5         NumStorePurchases    165.378180
10             AcceptedCmp5    107.219802
6              AcceptedCmp1     41.907566
11             AcceptedCmp6     22.354226
9              AcceptedCmp4     16.194989
0                 Education     15.059859
1                 Age_Group      5.420744
16  Marital_Status_Together      4.725712
15    Marital_Status_Single      3.301889
13  Marital_Status_Divorced     -0.070874
8              AcceptedCmp3     -1.329466
7              AcceptedCmp2     -3.573823
12                 Complain     -6.977055
14   Marital_Status_Married     -7.152616
3         NumDealsPurchases    -26.833526
2                Dependents    -87.447272


In [27]:
# Implementing L1 Regularization (Lasso Regression)
# Tuning Regularization parameter alpha
# sgdr = SGDRegressor(eta0=.0001, max_iter=10000, penalty='l1', random_state=1)
# grid_param = {'alpha': [0.1,1,10,25,50,75]}

# gd_sr = GridSearchCV(estimator=sgdr, param_grid=grid_param, scoring='r2', cv=5)

# gd_sr.fit(X_scaled, Y)

# best_parameters = gd_sr.best_params_
# print(best_parameters)

# best_result = gd_sr.best_score_ # Mean cross-validated score of the best_estimator
# print(best_result)

{'alpha': 10}
0.763863881423845


In [28]:
# Building SGDRegressor using the tuned parameters
# sgdr = SGDRegressor(eta0=.0001, max_iter=10000, penalty='l1', alpha=10, random_state=1)
# sgdr.fit(X_scaled,Y)
# print('Intercept', sgdr.intercept_)
# print(pd.DataFrame(zip(X.columns, sgdr.coef_), columns=['Features','Coefficients']).sort_values(by=['Coefficients'],ascending=False))

Intercept [603.18763717]
                   Features  Coefficients
4           NumWebPurchases    276.310984
5         NumStorePurchases    166.510779
10             AcceptedCmp5    105.043397
6              AcceptedCmp1     40.438732
11             AcceptedCmp6     17.748533
9              AcceptedCmp4      9.940631
0                 Education      8.536643
15    Marital_Status_Single      0.000000
13  Marital_Status_Divorced      0.000000
12                 Complain      0.000000
8              AcceptedCmp3      0.000000
1                 Age_Group      0.000000
7              AcceptedCmp2      0.000000
16  Marital_Status_Together      0.000000
14   Marital_Status_Married     -1.610946
3         NumDealsPurchases    -11.088993
2                Dependents    -92.782556


In [29]:
# Implementing Elastic Net Regularization (Elastic Net Regression)
# Tuning Regularization parameter alpha and l1_ratio
# sgdr = SGDRegressor(eta0=.0001, max_iter=10000, penalty='elasticnet', random_state=1)
# grid_param = {'alpha': [.0001, .001, .01, .1, 1,10,50,80,100],'l1_ratio':[0, 0.1, 0.3,0.5,0.7,0.9,1]}

# gd_sr = GridSearchCV(estimator=sgdr, param_grid=grid_param, scoring='r2', cv=5)

# gd_sr.fit(X_scaled, Y)

# best_parameters = gd_sr.best_params_
# print(best_parameters)

# best_result = gd_sr.best_score_ # Mean cross-validated score of the best_estimator
# print(best_result)


{'alpha': 10, 'l1_ratio': 1}
0.763863881423845


In [30]:
## Building SGDRegressor using the tuned parameters
# sgdr = SGDRegressor(eta0=.0001, max_iter=10000, penalty='elasticnet', alpha=10, l1_ratio=1, random_state=1)
# sgdr.fit(X_scaled,Y)
# print('Intercept', sgdr.intercept_)
# print(pd.DataFrame(zip(X.columns, sgdr.coef_), columns=['Features','Coefficients']).sort_values(by=['Coefficients'],ascending=False))



Intercept [603.18763717]
                   Features  Coefficients
4           NumWebPurchases    276.310984
5         NumStorePurchases    166.510779
10             AcceptedCmp5    105.043397
6              AcceptedCmp1     40.438732
11             AcceptedCmp6     17.748533
9              AcceptedCmp4      9.940631
0                 Education      8.536643
15    Marital_Status_Single      0.000000
13  Marital_Status_Divorced      0.000000
12                 Complain      0.000000
8              AcceptedCmp3      0.000000
1                 Age_Group      0.000000
7              AcceptedCmp2      0.000000
16  Marital_Status_Together      0.000000
14   Marital_Status_Married     -1.610946
3         NumDealsPurchases    -11.088993
2                Dependents    -92.782556


In [33]:
# Implementing Support Vector Regression
# Tuning the SVR parameters 'kernel', 'C', 'epsilon' and implementing cross-validation using Grid Search
# svr = SVR()
# grid_param = {'kernel': ['linear', 'poly', 'rbf', 'sigmoid'], 'C': [100,1000,10000], 'epsilon': [10,100,1000,10000]}

# gd_sr = GridSearchCV(estimator=svr, param_grid=grid_param, scoring='r2', cv=5)

# gd_sr.fit(X_scaled, Y)

# best_parameters = gd_sr.best_params_
# print("Optimal parameters:\n", best_parameters)

# best_result = gd_sr.best_score_ # Mean cross-validated score of the best_estimator
# print("Best mean cross-validated score:\n", best_result)

Optimal parameters:
 {'C': 1000, 'epsilon': 100, 'kernel': 'rbf'}
Best mean cross-validated score:
 0.7900950196853797


In [34]:
 # Implementing Random Forest Regression
# # Tuning the random forest parameter 'n_estimators' and implementing cross-validation using Grid Search
# rfr = RandomForestRegressor(criterion='mse', max_features='sqrt', random_state=1)
# grid_param = {'n_estimators': [100,200,300,450,500]}

# gd_sr = GridSearchCV(estimator=rfr, param_grid=grid_param, scoring='r2', cv=5)

# gd_sr.fit(X_scaled, Y)

# best_parameters = gd_sr.best_params_
# print("Optimal parameters:\n", best_parameters)

# best_result = gd_sr.best_score_ # Mean cross-validated score of the best_estimator
# print("Best mean cross-validated score:\n", best_result)



Optimal parameters:
 {'n_estimators': 450}
Best mean cross-validated score:
 0.8151919271879431


In [35]:
# Building random forest using the tuned parameter
# rfr = RandomForestRegressor(n_estimators=450, criterion='mse', max_features='sqrt', random_state=1)
# rfr.fit(X_scaled,Y)
# featimp = pd.Series(rfr.feature_importances_, index=list(X)).sort_values(ascending=False)
# print(featimp)

NumWebPurchases            0.391925
NumStorePurchases          0.238325
Dependents                 0.112061
AcceptedCmp5               0.067392
NumDealsPurchases          0.048406
AcceptedCmp1               0.031047
Education                  0.024601
AcceptedCmp6               0.018035
AcceptedCmp4               0.015687
Age_Group                  0.015059
Marital_Status_Married     0.007163
Marital_Status_Together    0.007050
AcceptedCmp3               0.006772
Marital_Status_Single      0.006478
Marital_Status_Divorced    0.005483
AcceptedCmp2               0.004168
Complain                   0.000348
dtype: float64


In [46]:
# X_ = final_data[['NumWebPurchases', 'NumStorePurchases', 'Dependents','AcceptedCmp5','NumDealsPurchases','AcceptedCmp1','Education','AcceptedCmp6','AcceptedCmp4','Age_Group']]

# feature_scaler = StandardScaler()
# X_scaled_ = feature_scaler.fit_transform(X_)

# # Tuning the random forest parameter 'n_estimators' and implementing cross-validation using Grid Search
# rfr = RandomForestRegressor(criterion='mse', max_features='sqrt', random_state=1)
# grid_param = {'n_estimators': [450,500,550,600,650,700,750]}

# gd_sr = GridSearchCV(estimator=rfr, param_grid=grid_param, scoring='r2', cv=5)

# gd_sr.fit(X_scaled_, Y)

# best_parameters = gd_sr.best_params_
# print("Optimal parameters:\n", best_parameters)

# best_result = gd_sr.best_score_ # Mean cross-validated score of the best_estimator
# print("Best mean cross-validated score:\n", best_result)

Optimal parameters:
 {'n_estimators': 700}
Best mean cross-validated score:
 0.8119280708478798


In [47]:
import statsmodels.api as sm
import plotly.graph_objs as go

In [49]:
# Implement linear regression model
final_data.info()
X_ = sm.add_constant(X_scaled)
est = sm.OLS(Y, X_) # OLS stands for 'ordinary least squares'; this function implements linear regression
model = est.fit()
print(model.summary()) # this prints out a table containing variables' coefficients, p-values, etc.

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2240 entries, 0 to 2239
Data columns (total 19 columns):
 #   Column                   Non-Null Count  Dtype
---  ------                   --------------  -----
 0   Education                2240 non-null   int64
 1   Age_Group                2240 non-null   int64
 2   Income_Group             2240 non-null   int64
 3   Dependents               2240 non-null   int64
 4   NumDealsPurchases        2240 non-null   int64
 5   NumWebPurchases          2240 non-null   int64
 6   NumStorePurchases        2240 non-null   int64
 7   AcceptedCmp1             2240 non-null   int64
 8   AcceptedCmp2             2240 non-null   int64
 9   AcceptedCmp3             2240 non-null   int64
 10  AcceptedCmp4             2240 non-null   int64
 11  AcceptedCmp5             2240 non-null   int64
 12  AcceptedCmp6             2240 non-null   int64
 13  Complain                 2240 non-null   int64
 14  Revenue                  2240 non-null   int64
 15  Mari